# Spacy text to sql

In [8]:
!pip install openai spacy python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [1]:
import spacy

spacy.cli.download("en_core_web_sm")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [30]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 7.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [5]:

from openai import OpenAI

# Load spaCy model
nlp = spacy.load("en_core_web_sm")  # 'md' model provides vectors for similarity

# Define schema terms
schema = {
    "customers": ["customer_id", "name", "email", "location"],
    "orders": ["order_id", "customer_id", "product_id", "quantity", "order_date"],
    "products": ["product_id", "name", "price"]
}

# Flatten the schema for matching
schema_terms = []
for table, columns in schema.items():
    schema_terms.append(table)
    schema_terms.extend([f"{table}.{col}" for col in columns])

print('schema_terms:', schema_terms)


schema_terms: ['customers', 'customers.customer_id', 'customers.name', 'customers.email', 'customers.location', 'orders', 'orders.order_id', 'orders.customer_id', 'orders.product_id', 'orders.quantity', 'orders.order_date', 'products', 'products.product_id', 'products.name', 'products.price']


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()


In [10]:
def get_relevant_items_spacy(query, top_k=3):
    # Parse query with spaCy
    query_doc = nlp(query)

    # Calculate similarity between query and schema terms
    relevance_scores = []
    for term in schema_terms:
        schema_doc = nlp(term)
        score = query_doc.similarity(schema_doc)
        relevance_scores.append((term, score))
    
    # Sort terms by relevance score
    relevance_scores = sorted(relevance_scores, key=lambda x: x[1], reverse=True)
    
    # Get top-k relevant terms
    relevant_items = relevance_scores[:top_k]
    return relevant_items

In [11]:
def generate_sql_query_spacy(query):
    # Get top relevant schema items
    relevant_items = get_relevant_items_spacy(query)
    
    # Construct prompt with relevant schema items
    relevant_schema_text = "\n".join([f"- {item[0]}" for item in relevant_items])
    prompt = (
        f"Given the following SQL database schema and the query request, generate an SQL query.\n\n"
        f"Schema:\n{relevant_schema_text}\n\n"
        f"Query request: {query}\n\n"
        "SQL Query:"
    )

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": "Sql for developer get data"
            }
        ]
    )

    # Extract and return SQL query from response
    sql_query = completion.choices[0].message
    return sql_query


In [12]:
# Example query
query = "Show me all orders placed by customers in New York"
sql_query = generate_sql_query_spacy(query)
print("Generated SQL Query:", sql_query)

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_64882/139854315.py:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  score = query_doc.similarity(schema_doc)


Generated SQL Query: ChatCompletionMessage(content="To retrieve all orders placed by customers located in New York, you can use a SQL query that joins the `orders` table with the `customers` table. Here's how the query might look:\n\n```sql\nSELECT o.*\nFROM orders o\nJOIN customers c ON o.customer_id = c.id\nWHERE c.city = 'New York';\n```\n\nIn this query:\n- `o.*` selects all columns from the `orders` table.\n- We join the `orders` table (`o`) with the `customers` table (`c`) on their respective customer ID fields.\n- The `WHERE` clause filters the results to include only those customers whose city is 'New York'. \n\nMake sure to adjust the column names in the `JOIN` clause according to your actual database schema if they are named differently.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [13]:
!pip install mysql-connector-python

In [4]:
# write code connect mysql database and pluck all schema table

import mysql.connector

# Establish the connection
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='root',
    database='eccubedb',
    port=13306
)

# Create a cursor object
cursor = conn.cursor()

# Execute the query to get all tables in the schema
cursor.execute("SHOW TABLES")

# Fetch all tables
tables = cursor.fetchall()
# Initialize the structure dictionary
structure = {}

# Iterate over the tables
for table in tables:
    table_name = table[0]
    # Execute the query to get all columns of the table
    cursor.execute(f"SHOW COLUMNS FROM {table_name}")

    # Fetch all columns
    columns = cursor.fetchall()
    
    # Add table to the structure dictionary
    structure[table_name] = {}
    
    # Iterate over the columns
    for column in columns:
        column_name = column[0]
        column_type = column[1]
        # Add column name and type to the table dictionary
        structure[table_name][column_name] = column_type

# Print the structure dictionary
print(structure['dcm_areas'])
    

{'id': 'int unsigned', 'name': 'varchar(255)', 'order': 'smallint', 'created_at': 'datetime', 'updated_at': 'datetime', 'deleted_at': 'datetime'}


In [6]:
# Flatten the schema for matching
schema_terms = []
for table, columns in structure.items():
    schema_terms.append(table)
    schema_terms.extend([f"{table}.{col} " for col in columns])

print('schema_terms:', schema_terms[:20])

schema_terms: ['dcm_areas', 'dcm_areas.id ', 'dcm_areas.name ', 'dcm_areas.order ', 'dcm_areas.created_at ', 'dcm_areas.updated_at ', 'dcm_areas.deleted_at ', 'dcm_atms', 'dcm_atms.id ', 'dcm_atms.shop_id ', 'dcm_atms.name ', 'dcm_atms.business_hour ', 'dcm_atms.order ', 'dcm_atms.created_at ', 'dcm_atms.updated_at ', 'dcm_atms.deleted_at ', 'dcm_auth_lock_temporary', 'dcm_auth_lock_temporary.id ', 'dcm_auth_lock_temporary.email ', 'dcm_auth_lock_temporary.times_of_fail ']


In [ ]:
!pip install langchain

In [57]:
!pip install langchain langchain-openai

In [ ]:
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


from langchain_core.pydantic_v1 import BaseModel, Field

class TableExtract(BaseModel):
    """Information table relevant to the query"""
    tables: Optional[list] = Field(None, description="Name tables relevant to the query")

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert database engineer. "
            "Now extract relevant tables to this query"
        ),
        ("human", "{text}"),
    ]
)

In [62]:
model = ChatOpenAI()

In [63]:
query = "top customer having highest order (count by total_payment) in this month"

chat1= prompt.format_messages(text=query)

model.invoke(chat1)


AIMessage(content="To extract relevant tables for this query, we would need tables related to customers, orders, and payments. Here's an example of the relevant tables and their columns that could be used for this query:\n\n1. Customers table:\n   - customer_id (Primary key)\n   - customer_name\n   - customer_email\n   - ...\n\n2. Orders table:\n   - order_id (Primary key)\n   - customer_id (Foreign key referencing Customers table)\n   - order_date\n   - ...\n\n3. Payments table:\n   - payment_id (Primary key)\n   - order_id (Foreign key referencing Orders table)\n   - total_payment\n   - payment_date\n   - ...\n\nWith these tables, we can write a query to find the top customer with the highest number of orders (count by total_payment) in the current month. The query might look something like this:\n\n```sql\nSELECT c.customer_id, c.customer_name, COUNT(o.order_id) AS order_count\nFROM Customers c\nJOIN Orders o ON c.customer_id = o.customer_id\nJOIN Payments p ON o.order_id = p.order_

In [66]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.9 MB/s eta 0:00:00a 0:00:01


In [13]:
from rapidfuzz import fuzz
class SchemaAnalyzer:
    def __init__(self, host, user, password, database, port):
        self.conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        self.cursor = self.conn.cursor()
        # self.nlp = spacy.load("en_core_web_md")
        self.schema_terms = self._get_schema_terms()

    def _get_schema_terms(self):
        self.cursor.execute("SHOW TABLES")
        tables = self.cursor.fetchall()
        schema_terms = []
        for table in tables:
            table_name = table[0]
            schema_terms.append(table_name)
            self.cursor.execute(f"SHOW COLUMNS FROM {table_name}")
            columns = self.cursor.fetchall()
            for column in columns:
                column_name = column[0]
                term_add = f"{table_name}.{column_name}"
                schema_terms.append(term_add)
        return schema_terms

    def get_relevant_items_spacy(self, query, top_k=3):
        relevance_scores = []
        for term in self.schema_terms:
            score = fuzz.partial_ratio(query, term)
            relevance_scores.append((term, score))
            # print('term:', term, 'score:', score)
        relevance_scores = sorted(relevance_scores, key=lambda x: x[1], reverse=True)
        relevant_items = relevance_scores[:top_k]
        return relevant_items

    def generate_sql_query_spacy(self, query):
        relevant_items = self.get_relevant_items_spacy(query, 6)
        print('relevant_items', relevant_items)
        relevant_schema_text = "\n".join([f"- {item[0]}" for item in relevant_items])
        # prompt = (
        #     f"Given the following SQL database schema and the query request, generate an SQL query.\n\n"
        #     f"Schema:\n{relevant_schema_text}\n\n"
        #     f"Query request: {query}\n\n"
        #     "SQL Query:"
        # )

        # # Assuming you have a client object for OpenAI's API
        # completion = client.chat.completions.create(
        #     model="gpt-4o",
        #     messages=[
        #         {"role": "system", "content": prompt},
        #         {
        #             "role": "user",
        #             "content": "Sql for developer get data"
        #         }
        #     ]
        # )

        # sql_query = completion.choices[0].message
        # return sql_query
        return 'alo alo'

    def close_connection(self):
        self.cursor.close()
        self.conn.close()


In [14]:
analyzer = SchemaAnalyzer(
    host='127.0.0.1',
    user='root',
    password='root',
    database='eccubedb',
    port=13306
)

query = "top customer having highest order (count by total_payment) in this month"
sql_query = analyzer.generate_sql_query_spacy(query)
print(sql_query)
analyzer.close_connection()

relevant_items [('dtb_payment', 81.81818181818181), ('pt_customer', 81.81818181818181), ('dtb_customer', 75.0), ('dtb_payment.id', 71.42857142857143), ('pt_customer.id', 69.23076923076923), ('dtb_customer.id', 66.66666666666667)]
alo alo
